In [1]:
pip install Pillow


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import os
# import xml.etree.ElementTree as ET
# from PIL import Image
# import shutil

# # Define paths based on your structure
# root_dir = "final_splitted_dataset"
# train_dir = os.path.join(root_dir, "train")
# test_dir = os.path.join(root_dir, "test")
# output_images_dir = os.path.join(root_dir, "images")
# output_xml_base_dir = os.path.join(root_dir, "xmls")  # Base XML directory

# # Ensure output directories exist
# os.makedirs(output_images_dir, exist_ok=True)
# os.makedirs(output_xml_base_dir, exist_ok=True)

# # Supported image formats (case-insensitive)
# VALID_EXTENSIONS = {".jpg", ".jpeg", ".png", ".bmp", ".tiff"}

# # Function to create Pascal VOC XML annotation
# def create_voc_xml(image_path, class_name, output_xml_dir):
#     try:
#         image = Image.open(image_path)
#         width, height = image.size
#     except Exception as e:
#         print(f"Skipping {image_path} due to error: {e}")
#         return

#     annotation = ET.Element("annotation")

#     folder = ET.SubElement(annotation, "folder")
#     folder.text = "images"

#     filename = ET.SubElement(annotation, "filename")
#     filename.text = os.path.basename(image_path)

#     size = ET.SubElement(annotation, "size")
#     width_elem = ET.SubElement(size, "width")
#     width_elem.text = str(width)
#     height_elem = ET.SubElement(size, "height")
#     height_elem.text = str(height)
#     depth = ET.SubElement(size, "depth")
#     depth.text = "3"  # Assuming RGB images

#     obj = ET.SubElement(annotation, "object")
#     name = ET.SubElement(obj, "name")
#     name.text = class_name

#     bndbox = ET.SubElement(obj, "bndbox")
#     xmin = ET.SubElement(bndbox, "xmin")
#     xmin.text = "0"
#     ymin = ET.SubElement(bndbox, "ymin")
#     ymin.text = "0"
#     xmax = ET.SubElement(bndbox, "xmax")
#     xmax.text = str(width)
#     ymax = ET.SubElement(bndbox, "ymax")
#     ymax.text = str(height)

#     # Ensure class-specific XML folder exists
#     class_xml_dir = os.path.join(output_xml_dir, class_name)
#     os.makedirs(class_xml_dir, exist_ok=True)

#     # Save XML inside the class-specific folder
#     xml_filename = os.path.join(class_xml_dir, os.path.splitext(os.path.basename(image_path))[0] + ".xml")
#     tree = ET.ElementTree(annotation)
#     tree.write(xml_filename)

# # Count processed images
# total_images = 0

# # Process train and test folders
# for dataset_type, dataset_path in [("train", train_dir), ("test", test_dir)]:
#     for class_name in os.listdir(dataset_path):
#         class_path = os.path.join(dataset_path, class_name)
#         if os.path.isdir(class_path):
#             for img in os.listdir(class_path):
#                 img_ext = os.path.splitext(img)[1].lower()
#                 if img_ext in VALID_EXTENSIONS:
#                     img_path = os.path.join(class_path, img)

#                     # Create unique filenames to prevent overwriting
#                     new_filename = f"{dataset_type}_{class_name}_{img}"
#                     new_img_path = os.path.join(output_images_dir, new_filename)

#                     try:
#                         # Copy the image to "images" folder
#                         shutil.copy(img_path, new_img_path)
#                         create_voc_xml(new_img_path, class_name, output_xml_base_dir)
#                         total_images += 1
#                     except Exception as e:
#                         print(f"Error processing {img_path}: {e}")

# print(f"Conversion complete! Processed {total_images} images.")
# print("Images are stored in 'final_splitted_dataset/images/' and XML files are organized by class inside 'final_splitted_dataset/xmls/'.")


Conversion complete! Processed 7545 images.
Images are stored in 'final_splitted_dataset/images/' and XML files are organized by class inside 'final_splitted_dataset/xmls/'.


In [1]:
pip show pillow

Name: pillow
Version: 10.4.0
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: 
Author-email: "Jeffrey A. Clark" <aclark@aclark.net>
License: HPND
Location: c:\Users\Lenovo\anaconda3\Lib\site-packages
Requires: 
Required-by: bokeh, datashader, imageio, matplotlib, scikit-image, sentence-transformers, streamlit
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import shutil

# Define dataset directories
root_dir = "./archive/NewaFoodDataset"
images_dir = os.path.join(root_dir, "images")
txt_dir = os.path.join(root_dir, "labels")  # Assuming TXT annotations are inside "labels"
xml_output_dir = os.path.join(root_dir, "xmls")  # Pascal VOC XML output folder

# Ensure XML output directory exists
os.makedirs(xml_output_dir, exist_ok=True)

# Supported image formats
VALID_EXTENSIONS = {".jpg", ".jpeg", ".png", ".bmp", ".tiff"}

# Function to create Pascal VOC XML file
def create_voc_xml(image_path, txt_path, class_dict, output_xml_dir):
    try:
        image = Image.open(image_path)
        width, height = image.size
    except Exception as e:
        print(f"Skipping {image_path} due to error: {e}")
        return

    # Read the TXT file (assuming YOLO format: class_id cx cy w h)
    with open(txt_path, "r") as file:
        lines = file.readlines()

    if not lines:
        print(f"Skipping {image_path} (no annotations found in {txt_path})")
        return

    annotation = ET.Element("annotation")

    folder = ET.SubElement(annotation, "folder")
    folder.text = "images"

    filename = ET.SubElement(annotation, "filename")
    filename.text = os.path.basename(image_path)

    size = ET.SubElement(annotation, "size")
    width_elem = ET.SubElement(size, "width")
    width_elem.text = str(width)
    height_elem = ET.SubElement(size, "height")
    height_elem.text = str(height)
    depth = ET.SubElement(size, "depth")
    depth.text = "3"  # Assuming RGB images

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            print(f"Skipping malformed annotation in {txt_path}: {line}")
            continue

        class_id, cx, cy, w, h = map(float, parts)
        class_name = class_dict.get(int(class_id), f"class_{int(class_id)}")  # Map class ID to name

        # Convert YOLO format (normalized) to Pascal VOC absolute values
        xmin = int((cx - w / 2) * width)
        ymin = int((cy - h / 2) * height)
        xmax = int((cx + w / 2) * width)
        ymax = int((cy + h / 2) * height)

        # Ensure bounding box is within image dimensions
        xmin, ymin, xmax, ymax = max(0, xmin), max(0, ymin), min(width, xmax), min(height, ymax)

        obj = ET.SubElement(annotation, "object")
        name = ET.SubElement(obj, "name")
        name.text = class_name

        bndbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(xmin)
        ET.SubElement(bndbox, "ymin").text = str(ymin)
        ET.SubElement(bndbox, "xmax").text = str(xmax)
        ET.SubElement(bndbox, "ymax").text = str(ymax)

    # Save XML inside the main "xmls/" directory
    xml_filename = os.path.join(output_xml_dir, os.path.splitext(os.path.basename(image_path))[0] + ".xml")
    tree = ET.ElementTree(annotation)
    tree.write(xml_filename)

# Load class dictionary (Mapping class IDs to Names)
class_dict = {
    0: "Baji", 1: "Bara", 2: "Bhutan", 3: "Bhuti", 4: "Buffalo Masu(Dakulaa)",
    5: "Channa", 6: "Dhau", 7: "Kachilaa", 8: "Lain tarkari", 9: "Lainachar",
    10: "Saag", 11: "Thoo", 12: "aloo", 13: "egg", 14: "sapumicha", 15: "yomari"
}  # Updated with your 16 food classes

# Count processed images
total_images = 0

# Process each image and its corresponding TXT file
for img_file in os.listdir(images_dir):
    img_ext = os.path.splitext(img_file)[1].lower()
    if img_ext in VALID_EXTENSIONS:
        image_path = os.path.join(images_dir, img_file)
        txt_filename = os.path.splitext(img_file)[0] + ".txt"
        txt_path = os.path.join(txt_dir, txt_filename)

        if os.path.exists(txt_path):
            create_voc_xml(image_path, txt_path, class_dict, xml_output_dir)
            total_images += 1
        else:
            print(f"Warning: No TXT annotation found for {img_file}")

print(f"✅ Conversion complete! Processed {total_images} images.")
print("📂 XML annotations are stored inside 'final_splitted_dataset/xmls/'.")


Skipping ./archive/NewaFoodDataset\images\aug_152_jpg.rf.f2c9f26723e85ed650dde7234cf4061c.jpg (no annotations found in ./archive/NewaFoodDataset\labels\aug_152_jpg.rf.f2c9f26723e85ed650dde7234cf4061c.txt)
Skipping ./archive/NewaFoodDataset\images\aug_168_jpg.rf.b3a9438ba3e8fb39ef6ac7a761396cfa.jpg (no annotations found in ./archive/NewaFoodDataset\labels\aug_168_jpg.rf.b3a9438ba3e8fb39ef6ac7a761396cfa.txt)
Skipping ./archive/NewaFoodDataset\images\aug_207_jpg.rf.87da9b8abdf464734ba9214b6030035a.jpg (no annotations found in ./archive/NewaFoodDataset\labels\aug_207_jpg.rf.87da9b8abdf464734ba9214b6030035a.txt)
Skipping ./archive/NewaFoodDataset\images\aug_291_jpg.rf.e88e5987fdec7ed99901e563a912023d.jpg (no annotations found in ./archive/NewaFoodDataset\labels\aug_291_jpg.rf.e88e5987fdec7ed99901e563a912023d.txt)
Skipping ./archive/NewaFoodDataset\images\aug_436_jpg.rf.d1177f9372b8540b86423c857689b930.jpg (no annotations found in ./archive/NewaFoodDataset\labels\aug_436_jpg.rf.d1177f9372b85

In [6]:
import PIL
print(PIL.__version__)

from PIL import Image
img = Image.new('RGB', (100, 100), color=(255, 0, 0))
img.show()


11.1.0


In [2]:
import os

# Define dataset directories
root_dir = "final_splitted_dataset"
train_dir = os.path.join(root_dir, "train")
test_dir = os.path.join(root_dir, "test")

# Supported image extensions
VALID_EXTENSIONS = {".jpg", ".jpeg", ".png", ".bmp", ".tiff"}

# Function to count images in a directory
def count_images(directory):
    total_images = 0
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            total_images += len([img for img in os.listdir(class_path) if os.path.splitext(img)[1].lower() in VALID_EXTENSIONS])
    return total_images

# Count images in train and test folders
train_count = count_images(train_dir)
test_count = count_images(test_dir)

# Print results
print(f"Total images in train set: {train_count}")
print(f"Total images in test set: {test_count}")
print(f"Grand total images in dataset: {train_count + test_count}")


Total images in train set: 6029
Total images in test set: 1516
Grand total images in dataset: 7545
